In [1]:
def setup_montgomery_curve():
    K = GF(101)
    A = K(2)
    B = K(1)
    E = EllipticCurve(K, [0, B, 0, A, 0])
    return K, E, A

def select_safe_points(E):
    while True:
        P = E.random_point()
        Q = E.random_point()
        if P != Q and P != -Q and P != E(0) and Q != E(0) and P[0] != Q[0]:
            return P, Q

def direct_poly(P, Q, R):
    x1 = (P + Q)[0]
    x2 = (P - Q)[0]
    return (R - x1)*(R - x2)

def kummer_poly(xP, xQ, A, R):
    if xP == xQ:
        raise ValueError("xP == xQ → division par 0 dans les formules")
    try:
        trace = 2*A + ((xP + xQ)*(xP*xQ - 1)) / (xP - xQ)
        product = ((xP*xQ - 1)^2) / ((xP - xQ)^2)
        return R^2 - trace*R + product
    except ZeroDivisionError:
        raise ValueError("Division par 0 dans la formule Kummer")

def compare_roots(f1, f2, base_field, extension=False):
    if extension:
        L.<z> = GF(base_field.order()^2)
        f1_ext = f1.change_ring(L)
        f2_ext = f2.change_ring(L)
        roots1 = set(f1_ext.roots(multiplicities=False))
        roots2 = set(f2_ext.roots(multiplicities=False))
    else:
        roots1 = set(f1.roots(multiplicities=False))
        roots2 = set(f2.roots(multiplicities=False))

    return roots1, roots2, roots1 == roots2

def run_comparison(extension=False):
    print("▶️ Comparaison brutale vs Kummer")

    K, E, A = setup_montgomery_curve()
    R.<X> = PolynomialRing(K)
    P, Q = select_safe_points(E)

    print(f"Points choisis :\n  P = {P}\n  Q = {Q}\n")

    try:
        f1 = direct_poly(P, Q, X)
        f2 = kummer_poly(P[0], Q[0], A, X)

        print(f"📐 f_direct : {f1}")
        print(f"📐 f_kummer : {f2}\n")

        r1, r2, equal = compare_roots(f1, f2, K, extension=extension)

        print(f"✅ Racines f1 : {r1}")
        print(f"✅ Racines f2 : {r2}")
        print(f"🎯 Racines identiques : {equal}")
    except Exception as e:
        print("❌ Erreur pendant le calcul :", e)


In [2]:
run_comparison()


▶️ Comparaison brutale vs Kummer
Points choisis :
  P = (64 : 43 : 1)
  Q = (37 : 9 : 1)

📐 f_direct : X^2 + X + 6
📐 f_kummer : X^2 + 97*X + 88

✅ Racines f1 : {24, 76}
✅ Racines f2 : {59, 46}
🎯 Racines identiques : False
